In [ ]:

# "mistralai/Mistral-7B-Instruct-v0.2
# "deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free"
# "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
# "Qwen/Qwen2-VL-72B-Instruct"
# "scb10x/scb10x-llama3-typhoon-v1-5x-4f316"


In [8]:
import pandas as pd
import json
import re

excel_file_path = "2025-03-21_01-45-48_FINAL_MOA_records.xlsx"
json_file_path = "../data/all_processed_facts.json"

df_excel = pd.read_excel(excel_file_path, sheet_name="Sheet1")
with open(json_file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)

# Function to clean the 'shark_offer' column and ensure it's in JSON/dictionary format
def clean_shark_offer(offer):
    if isinstance(offer, str):
        # Remove surrounding ```json and ``` if present
        offer = re.sub(r"```json\s*", "", offer)  # Remove leading ```json
        offer = re.sub(r"\s*```", "", offer)  # Remove trailing ```

        # Try converting to dictionary
        try:
            return json.loads(offer)  # Convert to dictionary if valid JSON
        except json.JSONDecodeError:
            return {"error": "Invalid JSON format", "raw_data": offer}  # Return error marker if not valid
    return offer  # If already a dict, return as is

# Clean the 'shark_offer' column
df_excel["shark_offer"] = df_excel["shark_offer"].apply(clean_shark_offer)

# Extract product details and product facts from JSON data
def extract_product_info(scenario):
    scenario_key = f"facts_{scenario}"
    if scenario_key in json_data:
        product_details = json_data[scenario_key].get("product_description", {})
        product_facts = json_data[scenario_key].get("facts", {})
        final_offer = json_data[scenario_key].get("final_offer", {})
    else:
        product_details, product_facts, final_offer = {}, {}, {}  # Default empty if not found
    return product_details, product_facts, final_offer

# Apply extraction function
df_excel["Product details"], df_excel["Product facts"], df_excel["final_offer"] = zip(*df_excel["scenario"].apply(extract_product_info))

# Select final columns
df_final = df_excel[["scenario", "model_identifier", "Product details", "Product facts", "shark_offer", "final_offer"]]

# Save to CSV
output_csv_path = "processed_shark_tank_data.csv"
df_final.to_csv(output_csv_path, index=False)



In [33]:
import os
import pandas as pd
import json
import concurrent.futures
import threading
import re
from together import Together
from collections import Counter
import time

class JudgeLLM:
    def __init__(self, api_key=None, models=None):
        self.api_key = api_key or os.getenv('TOGETHER_API_KEY')
        self.client = Together(api_key=self.api_key)
        self.models = models
        self.lock = threading.Lock()  # Added lock for thread safety
        print(f"[DEBUG] Initialized JudgeLLM with models: {self.models}")
    
    def generate_response(self, model, messages):
        print(f"[DEBUG] Generating response for model: {model}")
        try:
            start_time = time.time()  # Start timing
            completion = self.client.chat.completions.create(
                model=model,
                messages=messages,
                stream=False,
            )
            end_time = time.time()  # End timing

            inference_time = round(end_time - start_time, 3)  # Compute inference time
            response = completion.choices[0].message.content
            print(f"[DEBUG] Response received from {model} (Time Taken: {inference_time}s): {response[:200]}...")
            return model, response, inference_time
        except Exception as e:
            print(f"[ERROR] Error generating response from {model}: {str(e)}")
            return model, f"Error: {str(e)}", None
    
    def extract_json_from_response(self, response):
        try:
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                return json.loads(json_match.group())
        except json.JSONDecodeError:
            print("[ERROR] JSON decode failed. Returning raw response.")
        return {"reasoning": response, "rating of final_offer": "Invalid"}
    
    def ensemble_llm_debate(self, processed_data):
        print("[DEBUG] Starting multi-LLM debate...")

        system_prompt = (
            "You are a panel of expert venture capitalists analyzing an investment deal based on structured business data. "
            "You are a panel of expert venture capitalists analyzing the terms of an investment deal based on structured business data. "
            "You will evaluate key elements including the product details, the Shark LLM's proposed final offer, and the historical actual offer, analysing if the shark's offer is fair to the business on a scale of 1-10. "
            "When evaluating the offer, consider the context of the business and the investment terms. Also use your own knowledge and experience on what is a fair offer in the industry and investment landscape. "
            "If the shark offer is perceived equivalent to the actual offer, the score should be 5. "
            "If the actual situation did not gave an offer to the entrepreneur but the Shark LLM did, you should still consider if the offer is fair with respect to the context of the actual decision. "
            "If neither the Shark LLM nor the actual situation gave an offer, you can assign a score of 5 but still provide a reasoning if the shark's decision is fair to the entrepreneur. "
            "The data provided includes: \n"
            "- Scenario Name: The unique business pitch scenario. "
            "- Product Details: Information on the business product details. "
            "- Product Facts: Financial Information on the business and its offering. "
            "- Shark LLM Offer: The proposed investment terms made by the Shark LLM. "
            "- Actual Offer: The real historical investment terms given to this business. "
            "Each LLM first provides an independent evaluation, then critiques and refines each other's responses before reaching a consensus. "
            "Return the result in JSON format: "
            '{"reasoning": "Short summary of justification of your score", "rating of final_offer": score (1-10)}'
            "Reasoning is to justify the score given to the final offer, not the summary of the facts or pitch discussion."
        )

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": json.dumps(processed_data, indent=2, default=str)},
        ]

        results = []
        individual_scores = []
        initial_responses = {}
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_to_model = {executor.submit(self.generate_response, model, messages): model for model in self.models}
            
            for future in concurrent.futures.as_completed(future_to_model):
                model, response,  inference_time = future.result()
                initial_responses[model] = response
                response_json = self.extract_json_from_response(response)

                if isinstance(response_json.get("rating of final_offer"), (int, float)):
                    with self.lock:  # Prevent race conditions
                        individual_scores.append(response_json["rating of final_offer"])
                        results.append({
                            "Scenario": processed_data.get("scenario", "Unknown"),
                            "Shark Model": processed_data.get("model_identifier", "Unknown"),
                            "Judge Model": model,
                            "Initial Reasoning": response_json.get("reasoning", "Not Available"),
                            "Initial Score": response_json["rating of final_offer"],
                            "Refined Score": None,
                            "Final Consensus Score": None,
                            "Final Reasoning": None,
                            "Inference Time (s)": inference_time,
                        })

        critique_results = []
        for model, initial_response in initial_responses.items():
            critique_prompt = (
                "You are an expert VC evaluating a deal. Here are multiple analyses from different experts:\n\n"
                f"{json.dumps(initial_responses, indent=2)}\n\n"
                "Your task: Critique and refine your own response in light of these evaluations. "
                "If convinced by another argument, update your score. Otherwise, justify why your score remains the same."
                "Return the result in JSON format: "
                '{"final_reasoning": "Final refined evaluation", "updated_rating": score (1-10)}'
            )

            critique_messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": critique_prompt}]
            start_time = time.time()  # Start critique timing
            model, critique_response, critique_time = self.generate_response(model, critique_messages)
            end_time = time.time()  # End critique timing
            total_time = round(end_time - start_time, 3)  # Compute total critique time
            critique_json = self.extract_json_from_response(critique_response)
            
            if isinstance(critique_json.get("updated_rating"), (int, float)):
                critique_results.append({
                    "Scenario": processed_data.get("Scenario", "Unknown"),
                    "Shark Model": processed_data.get("model_identifier", "Unknown"),
                    "Judge Model": model,
                    "Refined Score": critique_json["updated_rating"],
                    "Final Consensus Score": None,
                    "Final Reasoning": critique_json.get("final_reasoning", "Not Available"),
                    "Inference Critique Time (s)": total_time,
                })

        final_scores = [result["Refined Score"] for result in critique_results if isinstance(result["Refined Score"], (int, float))]
        consensus_score = self.majority_vote(final_scores)
        print(f"[DEBUG] Final consensus score computed: {consensus_score}")

        for result, critique in zip(results, critique_results):
            result["Refined Score"] = critique["Refined Score"]
            result["Final Consensus Score"] = consensus_score
            result["Final Reasoning"] = critique["Final Reasoning"]
            result["Inference Critique Time (s)"] = critique["Inference Critique Time (s)"]

        return pd.DataFrame(results)

    def majority_vote(self, scores):
        return round(sum(scores) / len(scores), 2) if scores else "Not Available"
    
    def judge_scoring(self, processed_data):
        print("[DEBUG] Starting judge_scoring function...")
        return self.ensemble_llm_debate(processed_data)

In [ ]:
llm_models = ["deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free", "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free", "scb10x/scb10x-llama3-typhoon-v1-5x-4f316"]
judge = JudgeLLM(api_key=TOGETHERAI_API_KEY, models=llm_models)

csv_file = 'processed_shark_tank_data.csv'
processed_df = pd.read_csv(csv_file)
#1st 6 rows
sampled_df = processed_df[:1914]   

# Update main execution loop:
final_results = []
for _, row in sampled_df.iterrows():
    scenario_data = row.to_dict()
    result_df = judge.judge_scoring(scenario_data)  # Remove CSV parameter
    final_results.append(result_df)

# Single save after processing all rows
all_results_df = pd.concat(final_results, ignore_index=True)
all_results_df.to_csv("final_judge_results_v6.csv", index=False)

[DEBUG] Initialized JudgeLLM with models: ['deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free', 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free', 'scb10x/scb10x-llama3-typhoon-v1-5x-4f316']
[DEBUG] Starting judge_scoring function...
[DEBUG] Starting multi-LLM debate...
[DEBUG] Generating response for model: deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free
[DEBUG] Generating response for model: meta-llama/Llama-3.3-70B-Instruct-Turbo-Free
[DEBUG] Generating response for model: scb10x/scb10x-llama3-typhoon-v1-5x-4f316
[DEBUG] Response received from scb10x/scb10x-llama3-typhoon-v1-5x-4f316 (Time Taken: 2.426s): {"reasoning": "The Shark LLM's offer is fair considering the business's growth potential, unique product offering, and strong customer base. However, the actual outcome was no deal, as the Sharks beli...
[DEBUG] Response received from meta-llama/Llama-3.3-70B-Instruct-Turbo-Free (Time Taken: 10.683s): {"reasoning": "The Shark LLM's proposed offer is considered fair, with a valuation of $8